In [1]:
%%capture output
!pip install brian2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create -y -n opensim_env python=3.10
!source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && conda install -y -c opensim-org opensim
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/activation.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/muscle_sim.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/neural_simulations.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/plot_time_series.py



In [1]:
%%capture output
!unzip Model.zip

In [4]:
%load_ext autoreload
%autoreload 2

from brian2 import *
import numpy as np
import pandas as pd
import os
import subprocess
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from collections import defaultdict

from plot_time_series import plot_times_series, load_sto_file
from neural_simulations import run_neural_simulations
from activation import decode_spikes_to_activation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def Ramp_Hold(T_reaction, dt, v=0.2, t_ramp=0.1, t_hold=0.3, plot=False):

  time = np.arange(0, T_reaction, dt)

  stretch = np.piecewise(
      time,
      [time < t_ramp,
      (time >= t_ramp) & (time < t_hold),
      time >= t_hold],
      [lambda t: v * t,
      lambda t: v * t_ramp,
      lambda t: v * t_ramp - v * (t - t_hold)]
  )
  velocity = np.gradient(stretch, dt)
  if plot:
    plt.plot(time, stretch)
    plt.xlabel('Time (s)')
    plt.ylabel('Stretch (m)')
    plt.title('Ramp-Hold Stretch Profile')
    plt.show()
  return stretch, velocity

def Sinusoidal_stretch(dt,T_reaction, A=0.01, f=2, plot=False):

  time = np.arange(0, T_reaction, dt)
  stretch = A * np.sin(2 * np.pi * f * time)
  velocity = np.gradient(stretch, dt)
  if plot:
    plt.plot(time, stretch)
    plt.xlabel('Time (s)')
    plt.ylabel('Stretch (m)')
    plt.title('Sinusoidal Stretch Profile')
    plt.show()

  return stretch, velocity

In [13]:
t_stretching_afferent = 16*ms
t_motoneuron_muscle = 10*ms
t_reaction = t_stretching_afferent + t_motoneuron_muscle
n_loop = 1
dt = 0.1*ms

name_temporary_input_opensim = 'activation.npy'
name_temporary_output_opensim = 'muscle_output.sto'
name_output_spikes = 'spikes.json'
name_output_muscle='muscle.df'

neuron_population = {"Ia": 10, "II": 10, "exc": 10, "motor": 10}

# Initialize stretch and velocity arrays
#stretch_init = np.zeros(int(t_reaction/dt))
#velocity_init = np.zeros(int(t_reaction/dt))

stretch, velocity = Ramp_Hold(t_reaction, dt, v=0.2, t_ramp=0.1, t_hold=0.3)

data_muscle = []
data_spikes = {
    "Ia": defaultdict(list),
    "II": defaultdict(list),
    "MN": defaultdict(list)
}
for j in range(n_loop):

    spikes=run_neural_simulations(stretch, velocity, neuron_population,dt,t_reaction, w_run=500*uS, p_run=0.4 )
    for fiber_type, spikes_values in spikes.items():
      for neuron_id, spikes in spikes_values.items():
        data_spikes[fiber_type][neuron_id].extend(spikes)

    activations=decode_spikes_to_activation([[t/second for t in value] for key, value in spikes["MN"].items()], dt/second, t_reaction/second)
    mean_activation = np.mean(activations, axis=0)
    np.save(name_temporary_input_opensim, mean_activation)
    !source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && python muscle_sim.py --dt {dt_second} --T {t_reaction_second} --activation {name_activation}  --output {name_output_opensim}

    table = load_sto_file(name_temporary_output_opensim)[:-1] #don't include the last element

    # Create batch data dictionary
    batch_data_muscle = {
        **{f'activation_{i}': activations[i] for i in range(neuron_population['motor'])},
        'activation_mean': mean_activation,
        'stretch': table['stretch'].values,
        'velocity': table['velocity'].values,
        'fiber_length': table['fiber_length'].values,
        'fiber_velocity': table['fiber_velocity'].values
    }

    data_muscle.append(pd.DataFrame(batch_data_muscle))

# Create DataFrame and apply time shifts
df_muscle = pd.concat(data_muscle)

df_muscle['Time']=df_muscle.index*dt/second
# Save the DataFrame to CSV
df_muscle.to_csv(name_output_muscle, index=False)

# Plot time series
plot_times_series(name_output_spikes, name_output_muscle)

{'Ia': defaultdict(<class 'list'>, {0: [10.7 * msecond], 1: [21.1 * msecond, 21.9 * msecond], 2: [0.7 * msecond, 3. * msecond, 5.1 * msecond, 7.5 * msecond, 23.8 * msecond], 3: [0.8 * msecond, 10.6 * msecond, 14.8 * msecond, 15.9 * msecond, 19.7 * msecond], 4: [6.3 * msecond, 10.6 * msecond], 5: [1.1 * msecond], 6: [], 7: [], 8: [], 9: [9.4 * msecond]}), 'II': defaultdict(<class 'list'>, {0: [18. * msecond, 20.9 * msecond, 24.5 * msecond], 1: [2.9 * msecond, 17.2 * msecond], 2: [12.4 * msecond], 3: [], 4: [5.6 * msecond], 5: [3.5 * msecond, 13.6 * msecond], 6: [6.1 * msecond, 16.8 * msecond, 21.6 * msecond, 22.7 * msecond], 7: [200. * usecond, 7. * msecond, 13.3 * msecond], 8: [], 9: [17.9 * msecond]}), 'MN': defaultdict(<class 'list'>, {0: [13.1 * msecond, 17.7 * msecond, 18.4 * msecond, 19. * msecond, 19.6 * msecond, 22. * msecond, 22.8 * msecond, 23.7 * msecond, 25.3 * msecond], 1: [1.3 * msecond, 3.1 * msecond, 3.4 * msecond, 3.8 * msecond, 4.5 * msecond, 5.2 * msecond, 5.8 * mseco

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices